3. Write a program to demonstrate the working of the decision tree based ID3 algorithm.
Use an appropriate data set for building the decision tree and apply this knowledge
toclassify a new sample.

In [4]:
import pandas as pd
from pandas import DataFrame 
df_tennis = pd.read_csv('/content/ds5.csv')
df_tennis

,Sl.no:,outlook,Temp,Humidity,Wind,Play Tennis
0,1,Sunny,Hot,High,Weak,No
1,2,Sunny,Hot,High,Strong,No
2,3,Overcast,Hot,High,Weak,Yes
3,4,Rainy,Mild,High,Weak,Yes
4,5,Rainy,Cool,Normal,Weak,Yes
5,6,Rainy,Cool,Normal,Strong,No
6,7,Overcast,Cool,Normal,Strong,Yes
7,8,Sunny,Mild,High,Weak,No
8,9,Sunny,Cool,Normal,Weak,Yes
9,10,Rainy,Mild,Normal,Weak,Yes


In [8]:
#df_tennis.columns[5]
df_tennis.keys()[5]

'Play Tennis'

In [20]:
df_tennis=df_tennis.drop('Sl.no:', axis=1)

Entropy of the Training Data Set

In [21]:
def entropy(probs):  
    import math
    return sum( [-prob*math.log(prob, 2) for prob in probs] )

#Function to calulate the entropy of the given Data Sets/List with respect to target attributes
def entropy_of_list(a_list):  
    #print("A-list",a_list)
    from collections import Counter
    cnt = Counter(x for x in a_list)   # Counter calculates the propotion of class
   # print("\nClasses:",cnt)
    #print("No and Yes Classes:",a_list.name,cnt)
    num_instances = len(a_list)*1.0   # = 14
    print("\n Number of Instances of the Current Sub Class is {0}:".format(num_instances ))
    probs = [x / num_instances for x in cnt.values()]  # x means no of YES/NO
    print("\n Classes:",min(cnt),max(cnt))
    print(" \n Probabilities of Class {0} is {1}:".format(min(cnt),min(probs)))
    print(" \n Probabilities of Class {0} is {1}:".format(max(cnt),max(probs)))
    return entropy(probs) # Call Entropy :
    
# The initial entropy of the YES/NO attribute for our dataset.
print("\n  INPUT DATA SET FOR ENTROPY CALCULATION:\n", df_tennis['Play Tennis'])

total_entropy = entropy_of_list(df_tennis['Play Tennis'])

print("\n Total Entropy of PlayTennis Data Set:",total_entropy)


  INPUT DATA SET FOR ENTROPY CALCULATION:
 0      No
1      No
2     Yes
3     Yes
4     Yes
5      No
6     Yes
7      No
8     Yes
9     Yes
10    Yes
11    Yes
12    Yes
13     NO
Name: Play Tennis, dtype: object

 Number of Instances of the Current Sub Class is 14.0:

 Classes: NO Yes
 
 Probabilities of Class NO is 0.07142857142857142:
 
 Probabilities of Class Yes is 0.6428571428571429:

 Total Entropy of PlayTennis Data Set: 1.1981174211304033


Information Gain of Attributes

In [22]:

def information_gain(df, split_attribute_name, target_attribute_name, trace=0):
    print("Information Gain Calculation of ",split_attribute_name)
    '''
    Takes a DataFrame of attributes, and quantifies the entropy of a target
    attribute after performing a split along the values of another attribute.
    '''
    # Split Data by Possible Vals of Attribute:
    df_split = df.groupby(split_attribute_name)
   # for name,group in df_split:
    #    print("Name:\n",name)
     #   print("Group:\n",group)
    
    # Calculate Entropy for Target Attribute, as well as
    # Proportion of Obs in Each Data-Split
    nobs = len(df.index) * 1.0
   # print("NOBS",nobs)
    df_agg_ent = df_split.agg({target_attribute_name : [entropy_of_list, lambda x: len(x)/nobs] })[target_attribute_name]
    #print([target_attribute_name])
    #print(" Entropy List ",entropy_of_list)
    #print("DFAGGENT",df_agg_ent)
    df_agg_ent.columns = ['Entropy', 'PropObservations']
    #if trace: # helps understand what fxn is doing:
     #   print(df_agg_ent)
    
    # Calculate Information Gain:
    new_entropy = sum( df_agg_ent['Entropy'] * df_agg_ent['PropObservations'] )
    old_entropy = entropy_of_list(df[target_attribute_name])
    return old_entropy - new_entropy


print('Info-gain for Outlook is :'+str( information_gain(df_tennis, 'outlook', 'Play Tennis')),"\n")
print('\n Info-gain for Humidity is: ' + str( information_gain(df_tennis, 'Humidity', 'Play Tennis')),"\n")
print('\n Info-gain for Wind is:' + str( information_gain(df_tennis, 'Wind', 'Play Tennis')),"\n")
print('\n Info-gain for Temperature is:' + str( information_gain(df_tennis, 'Temp','Play Tennis')),"\n")

Information Gain Calculation of  outlook

 Number of Instances of the Current Sub Class is 4.0:

 Classes: Yes Yes
 
 Probabilities of Class Yes is 1.0:
 
 Probabilities of Class Yes is 1.0:

 Number of Instances of the Current Sub Class is 5.0:

 Classes: NO Yes
 
 Probabilities of Class NO is 0.2:
 
 Probabilities of Class Yes is 0.6:

 Number of Instances of the Current Sub Class is 5.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.4:
 
 Probabilities of Class Yes is 0.6:

 Number of Instances of the Current Sub Class is 14.0:

 Classes: NO Yes
 
 Probabilities of Class NO is 0.07142857142857142:
 
 Probabilities of Class Yes is 0.6428571428571429:
Info-gain for Outlook is :0.3617241393770686 

Information Gain Calculation of  Humidity

 Number of Instances of the Current Sub Class is 7.0:

 Classes: NO Yes
 
 Probabilities of Class NO is 0.14285714285714285:
 
 Probabilities of Class Yes is 0.42857142857142855:

 Number of Instances of the Current Sub Class is 7.0:

 Classes

ID3 Algorithm

In [23]:
def id3(df, target_attribute_name, attribute_names, default_class=None):
    
    ## Tally target attribute:
    from collections import Counter
    cnt = Counter(x for x in df[target_attribute_name])# class of YES /NO
    
    ## First check: Is this split of the dataset homogeneous?
    if len(cnt) == 1:
        return next(iter(cnt))  # next input data set, or raises StopIteration when EOF is hit.
    
    ## Second check: Is this split of the dataset empty?
    # if yes, return a default value
    elif df.empty or (not attribute_names):
        return default_class  # Return None for Empty Data Set
    
    ## Otherwise: This dataset is ready to be devied up!
    else:
        # Get Default Value for next recursive call of this function:
        default_class = max(cnt.keys()) #No of YES and NO Class
        # Compute the Information Gain of the attributes:
        gainz = [information_gain(df, attr, target_attribute_name) for attr in attribute_names] #
        index_of_max = gainz.index(max(gainz)) # Index of Best Attribute
        # Choose Best Attribute to split on:
        best_attr = attribute_names[index_of_max]
        
        # Create an empty tree, to be populated in a moment
        tree = {best_attr:{}} # Iniiate the tree with best attribute as a node 
        remaining_attribute_names = [i for i in attribute_names if i != best_attr]
        
        # Split dataset
        # On each split, recursively call this algorithm.
        # populate the empty tree with subtrees, which
        # are the result of the recursive call
        for attr_val, data_subset in df.groupby(best_attr):
            subtree = id3(data_subset,
                        target_attribute_name,
                        remaining_attribute_names,
                        default_class)
            tree[best_attr][attr_val] = subtree
        return tree

Predicting Attributes

In [24]:
# Get Predictor Names (all but 'class')
attribute_names = list(df_tennis.columns)
print("List of Attributes:", attribute_names) 
attribute_names.remove('Play Tennis') #Remove the class attribute 
print("Predicting Attributes:", attribute_names)

List of Attributes: ['outlook', 'Temp', 'Humidity', 'Wind', 'Play Tennis']
Predicting Attributes: ['outlook', 'Temp', 'Humidity', 'Wind']


Tree Construction


In [25]:
# Run Algorithm:
from pprint import pprint
tree = id3(df_tennis,'Play Tennis',attribute_names)
print("\n\nThe Resultant Decision Tree is :\n")
#print(tree)
pprint(tree)
attribute = next(iter(tree))
print("Best Attribute :\n",attribute)
print("Tree Keys:\n",tree[attribute].keys())

Information Gain Calculation of  outlook

 Number of Instances of the Current Sub Class is 4.0:

 Classes: Yes Yes
 
 Probabilities of Class Yes is 1.0:
 
 Probabilities of Class Yes is 1.0:

 Number of Instances of the Current Sub Class is 5.0:

 Classes: NO Yes
 
 Probabilities of Class NO is 0.2:
 
 Probabilities of Class Yes is 0.6:

 Number of Instances of the Current Sub Class is 5.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.4:
 
 Probabilities of Class Yes is 0.6:

 Number of Instances of the Current Sub Class is 14.0:

 Classes: NO Yes
 
 Probabilities of Class NO is 0.07142857142857142:
 
 Probabilities of Class Yes is 0.6428571428571429:
Information Gain Calculation of  Temp

 Number of Instances of the Current Sub Class is 4.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.25:
 
 Probabilities of Class Yes is 0.75:

 Number of Instances of the Current Sub Class is 4.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.5:
 
 Probabilities of Class Yes is 0.